In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
 # Use Pandas get_dummies to convert categorical data
df = pd.get_dummies(df)
df.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_disposition_CANDIDATE,koi_disposition_CONFIRMED,koi_disposition_FALSE POSITIVE
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,0,1,0
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,0,0,1
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,0,0,1
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,0,1,0
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714,0,1,0


## KOI (Kepler Object of Interest) Columns:
### koi_disposition (y-value) = Exoplanet Archive Disposition (Confirmed = 1)
#### koi_period = Orbital Period (Days)
#### koi_time0bk = Transit Epoch (Center of first detected transit)
#### koi_impact = Impact Parameter (Distance from center of planet/stellar disc)
#### koi_duration = Transit Duration (Hours-Between planet/star first/last contact)
#### koi_prad = Planetary Radius
#### koi_teq = Equilibrium Temperature (Kelvin)
#### koi_model_snr = Transit depth normalized by mean uncertainty

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_prad', 'koi_teq', 'koi_model_snr']]
selected_features.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_prad,koi_teq,koi_model_snr
0,54.418383,162.513840,0.586,4.50700,2.83,443,25.8
1,19.899140,175.850252,0.969,1.78220,14.60,638,76.3
2,1.736952,170.307565,1.276,2.40641,33.46,1395,505.6
3,2.525592,171.595550,0.701,1.65450,2.75,1406,40.9
4,4.134435,172.979370,0.762,3.14020,2.77,1160,40.2


# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
X = selected_features
y = df['koi_disposition_CONFIRMED'].values.reshape(-1, 1)
print(X.shape, y.shape)

(6991, 7) (6991, 1)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_prad,koi_teq,koi_model_snr
6122,6.768901,133.077240,0.150,3.61600,1.24,1017,10.8
6370,0.733726,132.020050,0.291,2.30900,0.86,1867,13.8
2879,7.652707,134.460380,0.970,79.89690,3.21,989,254.3
107,7.953547,174.662240,0.300,2.63120,2.25,696,38.4
29,4.959319,172.258529,0.831,2.22739,12.21,1103,696.5


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)

# Train the Model



In [11]:
classifier = LogisticRegression()
classifier

LogisticRegression()

In [12]:
 classifier.fit(X_train, y_train)

/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression()

In [13]:
print(f"Training Data Score: {classifier.score(X_train_minmax, y_train_minmax)}")
print(f"Testing Data Score: {classifier.score(X_test_minmax, y_test_minmax)}")

Training Data Score: 0.25100133511348466
Testing Data Score: 0.2768878718535469


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [14]:
# Create the GridSearchCV model
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [15]:
SVC(kernel='linear')

SVC(kernel='linear')

In [16]:
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [17]:
# Train the model with GridSearch
grid.fit(X_train_minmax, y_train_minmax)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] ................... C=1, gamma=0.0001, score=0.749, total=   0.2s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] ................... C=1, gamma=0.0001, score=0.749, total=   0.2s
[CV] C=1, gamma=0.0001 ...............................................


...

[CV] ................... C=50, gamma=0.005, score=0.749, total=   0.2s
[CV] C=50, gamma=0.005 ...............................................


/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] ................... C=50, gamma=0.005, score=0.749, total=   0.2s


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   12.5s finished
/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [18]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'gamma': 0.0001}
0.7489986828604487


In [19]:
 # Make predictions with the hypertuned model
predictions = grid.predict(X_test_minmax)

In [20]:
print(classification_report(y_test_minmax, predictions,
                            target_names=['koi_disposition_CONFIRMED', 'koi_period']))
print(classification_report(y_test_minmax, predictions,
                            target_names=['koi_disposition_CONFIRMED', 'koi_time0bk']))
print(classification_report(y_test_minmax, predictions,
                            target_names=['koi_disposition_CONFIRMED', 'koi_impact']))
print(classification_report(y_test_minmax, predictions,
                            target_names=['koi_disposition_CONFIRMED', 'koi_duration']))
print(classification_report(y_test_minmax, predictions,
                            target_names=['koi_disposition_CONFIRMED', 'koi_prad']))
print(classification_report(y_test_minmax, predictions,
                            target_names=['koi_disposition_CONFIRMED', 'koi_teq']))
print(classification_report(y_test_minmax, predictions,
                            target_names=['koi_disposition_CONFIRMED', 'koi_model_snr']))

                           precision    recall  f1-score   support

koi_disposition_CONFIRMED       0.72      1.00      0.84      1264
               koi_period       0.00      0.00      0.00       484

                 accuracy                           0.72      1748
                macro avg       0.36      0.50      0.42      1748
             weighted avg       0.52      0.72      0.61      1748

                           precision    recall  f1-score   support

koi_disposition_CONFIRMED       0.72      1.00      0.84      1264
              koi_time0bk       0.00      0.00      0.00       484

                 accuracy                           0.72      1748
                macro avg       0.36      0.50      0.42      1748
             weighted avg       0.52      0.72      0.61      1748

                           precision    recall  f1-score   support

koi_disposition_CONFIRMED       0.72      1.00      0.84      1264
               koi_impact       0.00      0.00      0.0

/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'serena_baker.sav'
joblib.dump(model, filename)

['serena_baker.sav']